In [2]:
import os
import pandas as pd
from tqdm import tqdm
import re
import langid 
import random
import numpy as np
import string
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
import spacy
import pickle
import json

from functions import *

stopwords_ = {}
stopwords_.update({'gb':stopwords.words('english') + "hon honorable government parliament gentleman gentlemen friend right minister".split(' ')})
stopwords_.update({'it':stopwords.words('italian') + "ministro ministra signore signori parlamento comitato governo presidente presidentessa segretario segretaria".split(' ')})
stopwords_.update({'nl':stopwords.words('dutch') + "minister parlement wij zij het motie moeten komen debat voorzitter nederland".split(' ')})

In [7]:
def train_lda(iso,k_range=[25,50,100]):

    for num_topics in k_range:
        df = data_loader.load_month(iso,"2020-03","2020-08")
        df['text'] = [utils.preprocess(str(x),stopwords_=stopwords_[iso]) for x in df['lemmatized']]
        texts = [x.split(' ') for x in df['text']]
        id2word = corpora.Dictionary(texts)
        corpus = [id2word.doc2bow(text) for text in texts]

        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,
                                                num_topics=num_topics,
                                                random_state=100,update_every=1,chunksize=100,
                                                passes=100,alpha='auto',
                                                per_word_topics=True)

        # Save model
        with open(f"/home/ruben/Documents/GitHub/ParlaMintCase/results/models/lda-{iso}-k{num_topics}-model.pkl",'wb') as f:
            pickle.dump(lda_model, f)

        # Save dictionary
        id2word.save(f"/home/ruben/Documents/GitHub/ParlaMintCase/results/models/lda-{iso}-k{num_topics}-dictionary.pkl")

        # Save corpus
        with open(f"/home/ruben/Documents/GitHub/ParlaMintCase/results/models/lda-{iso}-k{num_topics}-corpus.json",'w',encoding='utf-8') as f:
            json.dump(corpus,f)

In [8]:
for iso in ['gb','nl','it']:
    print(iso)
    train_lda(iso)

gb


KeyboardInterrupt: 